In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
import seaborn as sns
import matplotlib.pyplot as plt
import os
import gc
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import warnings
warnings.simplefilter('ignore')

In [16]:
def loadData(DataName):
    DataPath = os.path.join('./', DataName)
    if not os.path.exists(DataPath):
        print('%s does not exist!' % DataPath)
        return
    OriginData = pd.read_csv(DataPath)
    #OriginData = OriginData.sample(frac=1)  # 打乱顺序后返回
    return OriginData

In [17]:
import os
os.listdir()

['.ipynb_checkpoints',
 'ensall.csv',
 'ensall_0805.csv',
 'Ensemble.ipynb',
 'ens_0805.csv',
 'lgbm0805.csv',
 'lgbm_0805_singlemodel.csv',
 'mean20lgb.csv',
 'mean30lgb_0804.csv',
 'meanxgb0804.csv',
 'merge0805.csv',
 'merge0805_high.csv',
 'mergelgb0805.csv',
 'merge_0806.csv',
 'sub_ens_CV0800.csv',
 'sub_ens_cv08012.csv',
 'xgb0803']

In [18]:
r1 = loadData('merge0805.csv')

In [19]:
r1.head()

,SK_ID_CURR,TARGET
0,100001,0.475519
1,100005,0.876038
2,100013,0.436219
3,100028,0.522470
4,100038,0.871423


In [20]:
import os
os.listdir()

['.ipynb_checkpoints',
 'ensall.csv',
 'ensall_0805.csv',
 'Ensemble.ipynb',
 'ens_0805.csv',
 'lgbm0805.csv',
 'lgbm_0805_singlemodel.csv',
 'mean20lgb.csv',
 'mean30lgb_0804.csv',
 'meanxgb0804.csv',
 'merge0805.csv',
 'merge0805_high.csv',
 'mergelgb0805.csv',
 'merge_0806.csv',
 'sub_ens_CV0800.csv',
 'sub_ens_cv08012.csv',
 'xgb0803']

In [21]:
import glob
import pandas as pd
import numpy as np

# USING WEIGHTED AVERAGE RANK METHOD
# Plese refer this discussion for more detials - https://www.kaggle.com/c/home-credit-default-risk/discussion/60934

data = {}

for path in glob.glob("*.csv", recursive=True):
    data[path[:-4]] = pd.read_csv(path)

In [22]:
data.keys()

dict_keys(['ensall', 'ensall_0805', 'ens_0805', 'lgbm0805', 'lgbm_0805_singlemodel', 'mean20lgb', 'mean30lgb_0804', 'meanxgb0804', 'merge0805', 'merge0805_high', 'mergelgb0805', 'merge_0806', 'sub_ens_CV0800', 'sub_ens_cv08012'])

In [23]:
ranks = pd.DataFrame(columns=data.keys())
for key in data.keys():
    ranks[key] = data[key].TARGET.rank(method='min')

In [24]:
ranks['Average'] = ranks.mean(axis=1)
ranks['Scaled Rank'] = (ranks['Average'] - ranks['Average'].min()) / (ranks['Average'].max() - ranks['Average'].min())
ranks.corr()

,ensall,ensall_0805,ens_0805,lgbm0805,lgbm_0805_singlemodel,mean20lgb,mean30lgb_0804,meanxgb0804,merge0805,merge0805_high,mergelgb0805,merge_0806,sub_ens_CV0800,sub_ens_cv08012,Average,Scaled Rank
ensall,1.000000,0.995446,0.996728,0.995653,0.990964,0.991615,0.989911,0.991227,0.996319,0.996815,0.992975,0.998257,0.997062,0.998071,0.998360,0.998360
ensall_0805,0.995446,1.000000,0.998913,0.986704,0.995375,0.995960,0.997710,0.997267,0.990614,0.993754,0.998769,0.995728,0.998785,0.996475,0.999109,0.999109
ens_0805,0.996728,0.998913,1.000000,0.989777,0.993018,0.993621,0.994697,0.993799,0.992462,0.994627,0.997102,0.996754,0.999238,0.996553,0.998807,0.998807
lgbm0805,0.995653,0.986704,0.989777,1.000000,0.977526,0.978353,0.978153,0.981955,0.998020,0.996693,0.982955,0.996493,0.989143,0.987958,0.991791,0.991791
lgbm_0805_singlemodel,0.990964,0.995375,0.993018,0.977526,1.000000,0.999447,0.995104,0.993219,0.981339,0.984848,0.994714,0.988759,0.994882,0.995150,0.995013,0.995013
mean20lgb,0.991615,0.995960,0.993621,0.978353,0.999447,1.000000,0.995705,0.993748,0.982057,0.985547,0.995230,0.989465,0.995354,0.995679,0.995546,0.995546
mean30lgb_0804,0.989911,0.997710,0.994697,0.978153,0.995104,0.995705,1.000000,0.994307,0.982909,0.987114,0.998716,0.991023,0.994806,0.992982,0.995643,0.995643
meanxgb0804,0.991227,0.997267,0.993799,0.981955,0.993219,0.993748,0.994307,1.000000,0.987448,0.991464,0.994995,0.991199,0.994705,0.992633,0.995989,0.995989
merge0805,0.996319,0.990614,0.992462,0.998020,0.981339,0.982057,0.982909,0.987448,1.000000,0.999364,0.987218,0.997018,0.991925,0.990383,0.994492,0.994492
merge0805_high,0.996815,0.993754,0.994627,0.996693,0.984848,0.985547,0.987114,0.991464,0.999364,1.000000,0.990881,0.997867,0.994158,0.992092,0.996509,0.996509


In [60]:
ranks.corr().head()

,ensall,ensall_0805,ens_0805,lgbm0805,lgbm_0805_singlemodel,mean30lgb_0804,meanxgb0804,merge0805,merge0805_high,mergelgb0805,merge_0806,sub_ens_CV0800,Average,Scaled Rank
ensall,1.000000,0.997855,0.998648,0.994452,0.992111,0.993439,0.993504,0.995881,0.997286,0.996357,0.999395,0.998634,0.999584,0.999584
ensall_0805,0.997855,1.000000,0.998913,0.986704,0.995375,0.997710,0.997267,0.990614,0.993754,0.998769,0.995728,0.998785,0.999075,0.999075
ens_0805,0.998648,0.998913,1.000000,0.989777,0.993018,0.994697,0.993799,0.992462,0.994627,0.997102,0.996754,0.999238,0.998871,0.998871
lgbm0805,0.994452,0.986704,0.989777,1.000000,0.977526,0.978153,0.981955,0.998020,0.996693,0.982955,0.996493,0.989143,0.992420,0.992420
lgbm_0805_singlemodel,0.992111,0.995375,0.993018,0.977526,1.000000,0.995104,0.993219,0.981339,0.984848,0.994714,0.988759,0.994882,0.994011,0.994011


In [25]:
ranks['Score'] = ranks[['sub_ens_cv08012','lgbm0805']].mul([0.7,0.3]).sum(1) / ranks.shape[0]

In [26]:
ranks['Score'].head()

0    0.483606
1    0.850839
2    0.459041
3    0.494834
4    0.853629
Name: Score, dtype: float64

In [27]:
gender_submission = pd.DataFrame({'SK_ID_CURR':r1.SK_ID_CURR,'TARGET':ranks['Score']})
gender_submission.to_csv('ensall.csv', index = False)

In [28]:
subtmp = pd.read_csv('ensall.csv')
subtmp.head()

,SK_ID_CURR,TARGET
0,100001,0.483606
1,100005,0.850839
2,100013,0.459041
3,100028,0.494834
4,100038,0.853629


In [16]:
sub_lgb13_0804.head()

NameError: name 'sub_lgb13_0804' is not defined

In [ ]:
gender_submission = pd.DataFrame({'SK_ID_CURR':subtmp.SK_ID_CURR,'TARGET': ranks[['ens_0804','sub_lgb13_0804','sub_lgb7_0804',
                                                                                 'sub_lgb8_0804','sub_lgb9_0804']].mean(axis=1)/ ranks.shape[0]})
gender_submission.to_csv('testmerge0804.csv', index = False)

In [ ]:
gender_submission.head()

### 求xgb的均值


In [ ]:
def getMeanSub():
    ModelCount = 0
    submission_ensemble = pd.DataFrame()
    ResultSaveSubDirWithCategory = 'xgb0803'
    for SubDir in os.listdir(ResultSaveSubDirWithCategory):
        print(SubDir)
        submission = pd.read_csv(os.path.join(
            ResultSaveSubDirWithCategory, SubDir), index_col=0)
        if len(submission_ensemble):
            submission_ensemble['TARGET'] += submission['TARGET']
        else:
            submission_ensemble = submission.copy()
        ModelCount += 1
    submission_ensemble['TARGET'] /= ModelCount
    return submission_ensemble

In [ ]:
submission_ensemble = getMeanSub()

In [ ]:
submission_ensemble.head()

In [ ]:
submission_ensemble.to_csv('ens_xgb0803.csv')